In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
!pip install qiskit

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# pytorch 에서 gpu 사용 선택 (0번 아이디를 가진 gpu를 사용)
device = torch.device("cuda:0") 

In [ ]:
# 사전 훈련된 kobert model 불러옴
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 구글 드라이브에서 학습시킬 데이터 셋 가져오기
# /content/drive/MyDrive/ == 기본 경로
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
# 데이터 셋 크기 확인
len(chatbot_data)

In [ ]:
# 랜덤으로 샘플데이터 10개 출력
chatbot_data.sample(n=10)

In [ ]:
# 데이터 셋 같은 경우, 0~6 까지 7개의 분류로 구성되어 있음 
# 해당 데이터 셋의 공포, 놀람 등 항목을 사용하기 편하도록 숫자로 변경
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
# 학습 데이터 셋 구성
# 가져온 데이터 셋 파일에서 sentence와 emotion 항목을 q와 label로 각각 구성하여 학습을 위한 데이터 셋으로 구성 
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# 데이터 셋의 형태 확인
print(len(data_list))
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

In [ ]:
# sklearn.model_selection 패키지를 통하여 학습 데이터와 테스트 데이터로 분할
from sklearn.model_selection import train_test_split

# 75%는 학습 데이터셋, 25%는 테스트 데이터셋으로 구성
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
# 학습 데이터셋과 테스트 데이터셋의 분할 확인 및 형태 확인
print(len(dataset_train))
print(len(dataset_test))
print(dataset_train[0])
print(dataset_test[0])

In [ ]:
# 학습에 사용할 데이터 셋 클래스 선언
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        # transform하여 토큰화 하는 과정에서 
        # transform(i[인덱스]) == transform("아아아아 아아아 아아") 같은 경우 토큰화가 정상적으로 진행 안되는 오류 
        # transform([i[인덱스]]) == transform(["아아아아 아아아 아아"]) 형식으로 transform 진행
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    # 전체 데이터 셋에서 해당하는 인덱스의 sentences와 labels만 뽑는 함수
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    # 데이터 셋의 사이즈 return
    def __len__(self):
        return (len(self.labels))

In [ ]:
# 모델 파라미터 설정
# 토큰의 최대 길이라고 생각
max_len = 64
# 몇 개의 샘플들을 예측해보고 가중치를 업데이트 할 지 설정
# 아래와 같이 배치 사이즈가 64인 경우 데이터 64개 마다 예측한 것을 실제 값과 비교한다
batch_size = 64
warmup_ratio = 0.1
# epoch 횟수는 모델이 전체 데이터셋을 훈련시킬 횟수를 의미한다.
num_epochs = 10
max_grad_norm = 1
log_interval = 200
# learning_rate 값이 너무 크면 원하는 값에 도달하기 힘들고, 너무 작으면 학습기간이 오래 걸린다.
learning_rate =  5e-5

In [ ]:
# kobert에서 vocab을 통해서 토큰화 진행
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
# 학습 데이터셋과 테스트 데이터셋 토큰화 진행
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
# 토큰화 확인 (마지막에 호출된 데이터셋이 출력됨)
data_train[0]
# data_test[0]

In [ ]:
# DataLoader를 통해서 전체 데이터셋이 batch_size로 분할
# num_workers의 경우 높을수록 load 속도 상승
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
# 분류에 사용할 모델 클래스 선언
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 # num_classes는 카테고리의 개수를 의미한다. (현재 데이터셋의 경우 7개의 분류로 데이터셋이 구성되어 있음)
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# to(Device)는 위의 torch.device("cuda:0")를 의미하며 
# GPU에서 학습된 모델을 GPU로 불러올 때 사용한다.
# 또한 GPU로 학습된 모델에 데이터를 제공할 때도 to(Device)를 붙여줘야 함.
# bert모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# optimizer = 전체 데이터 셋의 실제 결과와 모델이 예측한 값 간의 차이가 효율적으로 좁혀질 수 있도록 최적화해주는 역할
# transform에서 제공하는 AdamW optimizer 사용
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# 예측값과 실제값의 오차인 손실함수는 pytorch에서 제공하는 다중분류를 위한 대표적인 손실함수인 torch.nn.CrossEntropyLoss 사용
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# 총 학습 수는 학습데이터의 크기 * 에폭수
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
# 시간이 지남에 따라 학습률을 조금씩 감소시키는 scheduler
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# 정확도 측정 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 모델 학습 
# 에폭수 만큼 반복
for e in range(num_epochs):
    # 정확도 초기화
    train_acc = 0.0
    test_acc = 0.0
    # 학습 모드
    model.train()

    # 배치만큼 학습 데이터셋 가져옴
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        # 그래디언트 = 순간변화율
        # 그래디언트 초기화
        optimizer.zero_grad()

        # 배치에서 데이터 추출
        # 모델에 제공하는 데이터이기때문에 to(device)
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # output
        out = model(token_ids, valid_length, segment_ids)

        # 손실 계산 (실제값과 예측값 간의 오차)
        loss = loss_fn(out, label)

        # 그래디언트에 값 더하기
        loss.backward()

        # 학습의 안정화를 위해 그래디언트 클리핑(자르기) => 기울기가 너무 커지는 것을 방지
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # 가중치 파라미터 업데이트
        optimizer.step()

        # 학습률 감소 스케줄러 
        scheduler.step()

        # 정확도
        train_acc += calc_accuracy(out, label)

        # batch 200 간격으로 학습상황 출력
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # 평가 모드
    model.eval()

    # 테스트 데이터셋을 통한 평가
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 구글 드라이브 기본 경로 = '/content/drive/MyDrive'
import os
# 해당 디렉토리로 이동
os.chdir('/content/drive/MyDrive/models')
# 현재 경로
os.getcwd()

In [ ]:
path = '/content/drive/MyDrive/models/'
# 지정한 path에 전체 모델 저장
torch.save(model, path + '7emotions_model.pt') 

# dict 저장
# torch.save(model.state_dict(), '7emotions_model_state_dict.pt')

# 필요한 값 지정해서 저장 가능
# torch.save({
#     'model': model.state_dict(),
#     'optimizer': optimizer.state_dict()
# }, '7emotions_all.tar')  